In [1]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - Basic Operation


### Install adk

In [2]:
%pip install --upgrade --quiet google-adk

In [3]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[2]
print(MODEL_NAME)

gemini-2.5-flash-preview-04-17


In [4]:
# @title Authentication to access to GCP
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

### Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.






In [5]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION


## Agent on adk

In [6]:
#@title Build a Agent with adk.
from google.adk.agents import LlmAgent
from google.adk.tools import google_search
from google.genai import types

PROMPT = """

You are a helpful assistant. Answer user questions using Google Search when needed.
Wirte a simple and clear answer to the user question.

"""

# https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
generate_content_config= types.GenerateContentConfig(
  temperature=0.2,
  max_output_tokens=250,
)

# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.agents.LlmAgent
agent = LlmAgent(
      name="Search_assistant",
      model=MODEL_NAME,
      instruction=PROMPT,
      generate_content_config = generate_content_config,
      description="An assistant that can search the web.",
      tools=[google_search] # Use Google search tool
)


## Query to Agent

In [7]:
from google.adk.sessions import InMemorySessionService, Session

APP_NAME = "Search_Assistant"
USER_ID = "shins"

# https://google.github.io/adk-docs/sessions/
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME,
                                         user_id=USER_ID,
                                         state={"initial_key": "initial_value"})


In [8]:
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

def call_agent(app_name:str,
               user_id:str,
               session_id:str,
               query:str,):

  # Runner : An in-memory Runner for testing and development.
  # https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.runners.Runner
  runner = Runner(agent=agent,
                  app_name=app_name,
                  session_service=session_service)

  # https://googleapis.github.io/python-genai/genai.html#genai.types.Content
  content = types.Content(role='user', parts=[types.Part(text=query)])

  # https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.runners.Runner.run
  events = runner.run(user_id=user_id,
                      session_id=session_id,
                      new_message=content)

  for event in events:
      if event.is_final_response():
          final_response = event.content.parts[0].text
          # final_response = event.content.parts[0]
          #print("Agent Response: \n", final_response)
          return final_response
      else:
          return "No results"



In [9]:

#QUERY = "What is the exchange rate from US dollars to Korean currency recently ?"
QUERY = "What is the situation of the world economy?"

call_agent(app_name = session.app_name,
           user_id = session.user_id,
           session_id = session.id,
           query=QUERY
)

"The global economic situation is characterized by steady but somewhat slow growth, moderating inflation, and persistent uncertainties.\n\nHere's a summary of the key points based on recent reports:\n\n*   **Growth:** Global growth is projected to remain around 3.1% to 3.2% in 2024 and 2025, similar to the pace in 2023 [1]. However, some projections indicate a potential slowdown to 2.3% in 2025, suggesting a possible recessionary trajectory driven by trade tensions and uncertainty [5]. The World Bank notes that the global economy seems to be settling at a low growth rate that may not be sufficient for sustained development [3]. Growth rates vary across regions, with a slight acceleration expected in advanced economies and a modest slowdown in emerging market and developing economies [1].\n*   **Inflation:** Global inflation is expected to continue declining from 2023 levels, with forecasts around 5.8% to 5.9% in 2024 and further decreases to around 4.4% to 4.5% in 2025 [1]. Advanced ec

## End of notebook